In [5]:
import os
import h5py
import numpy as np
h5_dir = '/data4/embedding/Post-NAT-BRCA/Virchow'
#h5_dir = '/data4/embedding/BCNB/CHIEF'
def read_assets_from_h5(h5_path: str):
    """Read the assets from the h5 file"""
    assets = {}
    attrs = {}
    try:
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
    except Exception as e:
        print(f'Error reading {h5_path}: {e}')
        return None, None
def read_assets_from_h5(h5_path: str):
    """Read the assets from the h5 file"""
    assets = {}
    attrs = {}
    try:
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
    except Exception as e:
        print(f'Error reading {h5_path}: {e}')
        return None, None
    
for file in os.listdir(h5_dir):
    if file.endswith('.h5'):
        #print(file)
        try:
            h5_file = os.path.join(h5_dir, file)
            assets, attrs = read_assets_from_h5(h5_file)

            #print(assets.keys())
            #print(assets['coords'].shape)
            print(assets['features'].shape)
        except:
            print(f'Error reading {h5_file}')
            break
            pass


(586, 1280)
(1569, 1280)
(557, 1280)
(844, 1280)
(1212, 1280)
(987, 1280)
(1057, 1280)
(509, 1280)
(694, 1280)
(1372, 1280)
(714, 1280)
(788, 1280)
(1381, 1280)
(922, 1280)
(881, 1280)
(721, 1280)
(596, 1280)
(1006, 1280)
(1243, 1280)
(655, 1280)
(492, 1280)
(589, 1280)
(1228, 1280)
(1322, 1280)
(1455, 1280)
(892, 1280)
(1190, 1280)
(1519, 1280)
(968, 1280)
(593, 1280)
(1213, 1280)
(637, 1280)
(646, 1280)
(561, 1280)
(944, 1280)
(907, 1280)
(671, 1280)
(1142, 1280)
(1037, 1280)
(778, 1280)
(1477, 1280)
(854, 1280)
(717, 1280)
(1009, 1280)
(1389, 1280)
(1266, 1280)
(1194, 1280)
(910, 1280)
(645, 1280)
(996, 1280)
(947, 1280)
(855, 1280)
(790, 1280)
(949, 1280)
(995, 1280)
(827, 1280)
(912, 1280)
(633, 1280)
(757, 1280)
(1063, 1280)
(614, 1280)
(863, 1280)
(525, 1280)
(752, 1280)
(726, 1280)
(1005, 1280)
(1009, 1280)
(1109, 1280)
(1093, 1280)
(1475, 1280)
(1535, 1280)
(930, 1280)
(1253, 1280)
(1207, 1280)
(1414, 1280)
(607, 1280)
(984, 1280)
(1253, 1280)
(1049, 1280)
(1205, 1280)
(869, 1

In [6]:
h5_dir = '/data4/embedding/AHSL/Gigapath/S18442.h5'
import h5py
def read_assets_from_h5(h5_path: str):
    """Read the assets from the h5 file"""
    assets = {}
    attrs = {}
    try:
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
    except Exception as e:
        print(f'Error reading {h5_path}: {e}')
        return None, None
assets, attrs = read_assets_from_h5(h5_dir)
assets.keys()

dict_keys(['mean_feature'])

In [30]:
import pandas as pd
file_path ='/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/biomarker/AIDPATH_RESTR.csv'
data = pd.read_csv(file_path)
data['label'].value_counts()

label
100    209
0      100
90      90
80      30
85      30
20      10
95      10
98      10
70      10
Name: count, dtype: int64

In [28]:
def get_tuning_methods(pretrain_model,model_type):
    if pretrain_model =='FMBC':
        return ["LR_Same_Patch"]
    # 如果模型类型为patch_level，则返回["LR"]，否则返回["LR"]
    return ["LR"] if model_type == "patch_level" else ["LR"]

learning_rates = [0.1,0.01,0.001, 0.0001]
tasks = {
    "TCGA-BRCA-SUBTYPE": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TCGA-BRCA-SUBTYPE.yaml"
    },
    "CAMELYON16_TEST_CANCER": {
        "embedding_dir": "/data4/embedding/CAMELYON16",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/CAMELYON16_TEST_CANCER.yaml"
    },
    "DORID_2": {
        "embedding_dir": "/data4/embedding/DORID",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/DORID_2.yaml"
    },
    "TCGA-BRCA_T": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TCGA-BRCA_T.yaml"
    },
    "BRACS_COARSE": {
        "embedding_dir": "/data4/embedding/BRACS",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/BRACS_COARSE.yaml"
    },
    "AHSL-NON-IDC-GRADE": {
        "embedding_dir": "/data4/embedding/AHSL",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/AHSL-NON-IDC-GRADE.yaml"
    },
    "CAMELYON16_TEST_IDC": {
        "embedding_dir": "/data4/embedding/CAMELYON16",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/CAMELYON16_TEST_IDC.yaml"
    },
    "BCNB_TUMOR": {
        "embedding_dir": "/data4/embedding/BCNB",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/BCNB_TUMOR.yaml"
    },
    "BCNB_ALN": {
        "embedding_dir": "/data4/embedding/BCNB",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/BCNB_ALN.yaml"
    },
    "AIDPATH_IDC": {
        "embedding_dir": "/data4/embedding/AIDPATH",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/AIDPATH_IDC.yaml"
    },
    "CAMELYON17_STAGE_4SUBTYPING": {
        "embedding_dir": "/data4/embedding/CAMELYON17",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/CAMELYON17_STAGE_4SUBTYPING.yaml"
    },
    "IMPRESS_RESIDUAL-TUMOR": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/IMPRESS_RESIDUAL-TUMOR.yaml"
    },
    "AHSL-GRADE-3": {
        "embedding_dir": "/data4/embedding/AHSL",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/AHSL-GRADE-3.yaml"
    },
    "BRACS_FINE": {
        "embedding_dir": "/data4/embedding/BRACS",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/BRACS_FINE.yaml"
    },
    "AHSL-GRADE-1": {
        "embedding_dir": "/data4/embedding/AHSL",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/AHSL-GRADE-1.yaml",
        "folds": 1,
    },
    # "BACH_TUMOR": {
    #     "embedding_dir": "/data4/embedding/BACH",
    #     "csv_dir": "dataset_csv/subtype",
    #     "task_cfg": "task_configs/subtype/BACH_TUMOR.yaml"
    # },
    "SLNBREAST_SUBTYPE": {
        "embedding_dir": "/data4/embedding/SLN-Breast",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/SLNBREAST_SUBTYPE.yaml"
    },
    "TCGA-BRCA_STAGE": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TCGA-BRCA_STAGE.yaml"
    },
    "POST-NAT-BRCA-3TYPE": {
        "embedding_dir": "/data4/embedding/Post-NAT-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/POST-NAT-BRCA-3TYPE.yaml",
        "folds":1
    },
    "TCGA-BRCA_M": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TCGA-BRCA_M.yaml"
    },
    "DORID_6": {
        "embedding_dir": "/data4/embedding/DORID",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/DORID_6.yaml"
    },
    "CPTAC_IDC": {
        "embedding_dir": "/data4/embedding/CPTAC",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/CPTAC_IDC.yaml"
    },
    "TUPAC_TUMOR_SCORE": {
        "embedding_dir": "/data4/embedding/TUPAC",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TUPAC_TUMOR_SCORE.yaml"
    },
    "AIDPATH_GRADE": {
        "embedding_dir": "/data4/embedding/AIDPATH",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/AIDPATH_GRADE.yaml"
    },
    "TCGA-BRCA_N": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/TCGA-BRCA_N.yaml"
    },
    "POST-NAT-BRCA-HERIHC": {
        "embedding_dir": "/data4/embedding/Post-NAT-BRCA",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/POST-NAT-BRCA-HERIHC.yaml",
        "folds":1
    },
    "CAMELYON17_STAGE": {
        "embedding_dir": "/data4/embedding/CAMELYON17",
        "csv_dir": "dataset_csv/subtype",
        "task_cfg": "task_configs/subtype/CAMELYON17_STAGE.yaml"
    },
    "TCGA-BRCA_MOLECULAR_SUBTYPING": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/TCGA-BRCA_MOLECULAR_SUBTYPING.yaml"
    },
    "IMPRESS_PCR": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_PCR.yaml"
    },
    "IMPRESS_PD-L1-TUMOR": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_PD-L1-TUMOR.yaml"
    },
    "IMPRESS_CD8-PERITUMORAL": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_CD8-PERITUMORAL.yaml"
    },
    "IMPRESS_PD-L1-STROMA": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_PD-L1-STROMA.yaml"
    },
    "IMPRESS_PR": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_PR.yaml"
    },
    "IMPRESS_CD163-PERITUMORAL": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_CD163-PERITUMORAL.yaml"
    },
    "BCNB_HER2": {
        "embedding_dir": "/data4/embedding/BCNB",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/BCNB_HER2.yaml"
    },
    "AIDPATH_KI67PRED": {
        "embedding_dir": "/data4/embedding/AIDPATH",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/AIDPATH_KI67PRED.yaml"
    },
    "POST-NAT-BRCA-PR": {
        "embedding_dir": "/data4/embedding/Post-NAT-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/POST-NAT-BRCA-PR.yaml",
        "folds":1
    },
    "BCNB_ER": {
        "embedding_dir": "/data4/embedding/BCNB",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/BCNB_ER.yaml"
    },
    "AIDPATH_CERB2": {
        "embedding_dir": "/data4/embedding/AIDPATH",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/AIDPATH_CERB2.yaml"
    },
    "IMPRESS_CD163-INTRATUMORAL": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_CD163-INTRATUMORAL.yaml"
    },
    "BCNB_PR": {
        "embedding_dir": "/data4/embedding/BCNB",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/BCNB_PR.yaml"
    },
    "POST-NAT-BRCA-ER": {
        "embedding_dir": "/data4/embedding/Post-NAT-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/POST-NAT-BRCA-ER.yaml",
        "folds":1
    },
    "IMPRESS_CD8-INTRATUMORAL": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_CD8-INTRATUMORAL.yaml"
    },
    "AIDPATH_RESTR": {
        "embedding_dir": "/data4/embedding/AIDPATH",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/AIDPATH_RESTR.yaml"
    },
    "POST-NAT-BRCA-ANTIHER2": {
        "embedding_dir": "/data4/embedding/Post-NAT-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/POST-NAT-BRCA-ANTIHER2.yaml",
        "folds":1
    },
    "CPTAC_AJCC8SUBTYPE": {
        "embedding_dir": "/data4/embedding/CPTAC",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/CPTAC_AJCC8SUBTYPE.yaml",
        "folds": 1
    },
    "IMPRESS_ER": {
        "embedding_dir": "/data4/embedding/IMPRESS",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/IMPRESS_ER.yaml"
    },
    "TCGA-BRCA_TP53": {
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/TCGA-BRCA_TP53.yaml"
    },
    "TCGA-BRCA_HRD": {
        "embedding_dir": "/data4/embedding/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/biomarker",
        "task_cfg": "task_configs/biomarker/TCGA-BRCA_HRD.yaml"
    },
    'MULTI_OMICS_ASCAT-PURITY':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS_ASCAT-PURITY.yaml"
    },
    'MULTI_OMICS_FRACTION_CANCER':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS_FRACTION_CANCER.yaml"
    },
    'MULTI_OMICS_ASCAT-PURITY':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS_ASCAT-PURITY.yaml"
    },
    'MULTI_OMICS_HRD':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS_HRD.yaml"
    },
    'MULTI_OMICS_IPS':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS_IPS.yaml"
    },
    'MULTI_OMICS':{
        "embedding_dir": "/data4/embedding/Multi-omic",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/MULTI_OMICS.yaml"
    },
    'TCGA-BRCA-GENE-EXP':{
        "embedding_dir": "/data4/embedding/TCGA-BRCA",
        "csv_dir": "dataset_csv/gene_exp",
        "task_cfg": "task_configs/gene_exp/TCGA-BRCA-GENE-EXP.yaml"
    }
}


len(tasks)
import yaml
for task,config_path in tasks.items():
    config = config_path['task_cfg']
    #read yaml
    with open(config, 'r') as f:
        config = yaml.safe_load(f)
        #print(config['name'])
        if task != config['name']:
            #print(task,config['name'])
            print(task,config_path['task_cfg'])